In [18]:
# https://youtu.be/78Iclo7f-zg
import numpy as np
from numpy import ma
import math
import scipy
import cv2
import matplotlib
from scipy.ndimage.filters import gaussian_filter
from config_reader import config_reader
import caffe
import util

cap = cv2.VideoCapture("video.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')

ret, frame = cap.read()
print(frame.shape)
out = cv2.VideoWriter('output.avi',fourcc, 23.0, (frame.shape[1],frame.shape[0]))

param, model = config_reader()
multiplier = [x * model['boxsize'] / frame.shape[0] for x in param['scale_search']]

len_multiplier = len(multiplier)
if param['use_gpu']: 
    print("gpu mode")
    caffe.set_mode_gpu()
    caffe.set_device(param['GPUdeviceNumber']) # set to your device!
else:
    print("cpu mode")
    caffe.set_mode_cpu()
    
net = caffe.Net(model['deployFile'], model['caffemodel'], caffe.TEST)

(720, 1280, 3)
gpu mode


In [19]:
# find connection in the specified sequence, center 29 is in the position 15
limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]
# the middle joints heatmap correpondence
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22], \
          [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], \
          [55,56], [37,38], [45,46]]
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
heatmap_avg = np.zeros((frame.shape[0], frame.shape[1], 19))    
paf_avg = np.zeros((frame.shape[0], frame.shape[1], 38))

def make_Realtime_Multi_Person_Pose_Estimation(oriImg,heatmap_avg,paf_avg):
    for m in range(len_multiplier):
        scale = multiplier[m]
        imageToTest = cv2.resize(oriImg, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model['stride'], model['padValue'])
        #print(imageToTest_padded.shape)
    
        #axarr[m].imshow(imageToTest_padded[:,:,[2,1,0]])
        #axarr[m].set_title('Input image: scale %d' % m)

        net.blobs['data'].reshape(*(1, 3, imageToTest_padded.shape[0], imageToTest_padded.shape[1]))
        #net.forward() # dry run
        net.blobs['data'].data[...] = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,2,0,1))/256 - 0.5;
        #start_time = time.time()
        output_blobs = net.forward()
        #print('At scale %d, The CNN took %.2f ms.' % (m, 1000 * (time.time() - start_time)))

        # extract outputs, resize, and remove padding
        heatmap = np.transpose(np.squeeze(net.blobs[ list(output_blobs.keys())[1]].data), (1,2,0)) # output 1 is heatmaps    
        heatmap = cv2.resize(heatmap, (0,0), fx=model['stride'], fy=model['stride'], interpolation=cv2.INTER_CUBIC)    
        heatmap = heatmap[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]    
        heatmap = cv2.resize(heatmap, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)
    
        paf = np.transpose(np.squeeze(net.blobs[list(output_blobs.keys())[0]].data), (1,2,0)) # output 0 is PAFs
        paf = cv2.resize(paf, (0,0), fx=model['stride'], fy=model['stride'], interpolation=cv2.INTER_CUBIC)
        paf = paf[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]
        paf = cv2.resize(paf, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)
    
        # visualization
        #axarr2[m].imshow(oriImg[:,:,[2,1,0]])
        #ax2 = axarr2[m].imshow(heatmap[:,:,3], alpha=.5) # right wrist
        #axarr2[m].set_title('Heatmaps (Rwri): scale %d' % m)
    
        #axarr3.flat[m].imshow(oriImg[:,:,[2,1,0]])
        #ax3x = axarr3.flat[m].imshow(paf[:,:,16], alpha=.5) # right elbow
        #axarr3.flat[m].set_title('PAFs (x comp. of Rwri to Relb): scale %d' % m)
        #axarr3.flat[len_multiplier + m].imshow(oriImg[:,:,[2,1,0]])
        #ax3y = axarr3.flat[len_multiplier + m].imshow(paf[:,:,17], alpha=.5) # right wrist
        #axarr3.flat[len_multiplier + m].set_title('PAFs (y comp. of Relb to Rwri): scale %d' % m)
          
        heatmap_avg = heatmap_avg + heatmap / len_multiplier
        paf_avg = paf_avg + paf / len_multiplier
  
    all_peaks = []
    peak_counter = 0
    
    for part in range(19-1):
        x_list = []
        y_list = []
        map_ori = heatmap_avg[:,:,part]
        map = gaussian_filter(map_ori, sigma=3)
    
        map_left = np.zeros(map.shape)
        map_left[1:,:] = map[:-1,:]
        map_right = np.zeros(map.shape)
        map_right[:-1,:] = map[1:,:]
        map_up = np.zeros(map.shape)
        map_up[:,1:] = map[:,:-1]
        map_down = np.zeros(map.shape)
        map_down[:,:-1] = map[:,1:]
    
        peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up, map>=map_down, map > param['thre1']))
        
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])) # note reverse
        
        peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    
    #if peak_counter < 17:
    #    return oriImg
        
    connection_all = []
    special_k = []
    
    #special_non_zero_index = []
    mid_num = 10
    
    for k in range(len(mapIdx)):
        score_mid = paf_avg[:,:,[x-19 for x in mapIdx[k]]]
        candA = all_peaks[limbSeq[k][0]-1]
        candB = all_peaks[limbSeq[k][1]-1]
        nA = len(candA)
        nB = len(candB)
        indexA, indexB = limbSeq[k]
        if(nA != 0 and nB != 0):
            connection_candidate = []
            for i in range(nA):
                for j in range(nB):
                    vec = np.subtract(candB[j][:2], candA[i][:2])
                    norm = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                    vec = np.divide(vec, norm)
                
                    startend = list(zip(np.linspace(candA[i][0], candB[j][0], num=mid_num), \
                               np.linspace(candA[i][1], candB[j][1], num=mid_num)))
                
                    vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0] for I in range(len(startend))])
                    vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1] for I in range(len(startend))])

                    score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                    if len(score_midpts) ==0:
                        a = 0
                    else:
                        a = sum(score_midpts)/len(score_midpts)
                    #(len(score_midpts) == 0 and 0 or sum(score_midpts)/len(score_midpts)) 
                    if norm == 0:
                        b = 0.0
                    else:
                        b = min(0.5*oriImg.shape[0]/norm-1, 0)
                    #(norm-1 == 0 and 0 or min(0.5*oriImg.shape[0]/norm-1, 0))
                    
                    score_with_dist_prior = a + b
                    criterion1 = len(np.nonzero(score_midpts > param['thre2'])[0]) > 0.8 * len(score_midpts)
                    criterion2 = score_with_dist_prior > 0
                    if criterion1 and criterion2:
                        connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])

            connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
            connection = np.zeros((0,5))
            for c in range(len(connection_candidate)):
                i,j,s = connection_candidate[c][0:3]
                if(i not in connection[:,3] and j not in connection[:,4]):
                    connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                    if(len(connection) >= min(nA, nB)):
                        break

            connection_all.append(connection)
        elif(nA != 0 or nB != 0):
            special_k.append(k)
            #special_non_zero_index.append(indexA if nA != 0 else indexB)
            connection_all.append([])
    
    # last number in each row is the total parts number of that person
    # the second last number in each row is the score of the overall configuration    
    subset = -1 * np.ones((0, 20))
    candidate = np.array([item for sublist in all_peaks for item in sublist])
    
    #print(special_k)
    #print(len(connection_all))
    #print(connection_all)    
    #connection_all_n = connection_all[0]
    #print(len(connection_all_n))
    #print(connection_all_n)
    
    #19개 인덱스가 아니면 사람이 아니라 판단하여 리턴
    if len(connection_all) != 19:
        print("connection_all size is not 19 : return")
        return oriImg
    
    #special_k가 없을 경우 return
    #if len(special_k) == 0:
    #    print("shape is zero : return")
    #    return oriImg
    #else:    #잡힌 값이 없으니 계산 생략하고 return
    #if special_k[0]==0:
    #    print("first shape size is zero : return")
    #    return oriImg        
    
    for k in range(len(mapIdx)):
        if k not in special_k:            
            partAs = connection_all[k][:,0]
            partBs = connection_all[k][:,1]
            indexA, indexB = np.array(limbSeq[k]) - 1

            for i in range(len(connection_all[k])): #= 1:size(temp,1)
                found = 0
                subset_idx = [-1, -1]
                for j in range(len(subset)): #1:size(subset,1):
                    if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                        subset_idx[found] = j
                        found += 1
            
                if found == 1:
                    j = subset_idx[0]
                    if(subset[j][indexB] != partBs[i]):
                        subset[j][indexB] = partBs[i]
                        subset[j][-1] += 1
                        subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
                elif found == 2: # if found 2 and disjoint, merge them
                    j1, j2 = subset_idx
                    #print "found = 2"
                    membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                    if len(np.nonzero(membership == 2)[0]) == 0: #merge
                        subset[j1][:-2] += (subset[j2][:-2] + 1)
                        subset[j1][-2:] += subset[j2][-2:]
                        subset[j1][-2] += connection_all[k][i][2]
                        subset = np.delete(subset, j2, 0)
                    else: # as like found == 1
                        subset[j1][indexB] = partBs[i]
                        subset[j1][-1] += 1
                        subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(20)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    row[-1] = 2
                    row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                    subset = np.vstack([subset, row])
    
    deleteIdx = [];
    for i in range(len(subset)):
        if subset[i][-1] < 4 or subset[i][-2]/subset[i][-1] < 0.4:
            deleteIdx.append(i)
    subset = np.delete(subset, deleteIdx, axis=0)
    
    cmap = matplotlib.cm.get_cmap('hsv')
    #subset is people count    
    for i in range(18):
        rgba = np.array(cmap(1 - i/18. - 1./36))
        rgba[0:3] *= 255
        for j in range(len(all_peaks[i])):
            cv2.circle(oriImg, all_peaks[i][j][0:2], 4, colors[i], thickness=-1)
        
    stickwidth = 4

    for i in range(17):
        for n in range(len(subset)):
            index = subset[n][np.array(limbSeq[i])-1]
            if -1 in index:
                continue
            cur_canvas = oriImg.copy()
            Y = candidate[index.astype(int), 0]
            X = candidate[index.astype(int), 1]
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
            oriImg = cv2.addWeighted(oriImg, 0.4, cur_canvas, 0.6, 0)
    
    return oriImg

In [20]:
img = make_Realtime_Multi_Person_Pose_Estimation(frame,heatmap_avg,paf_avg)
count = 0
while(cap.isOpened()):
    # Capture frame-by-frame
    
    ret, frame = cap.read()    
    if ret==True:
        out.write(make_Realtime_Multi_Person_Pose_Estimation(frame,heatmap_avg,paf_avg))        
        #print(count)
    else:
        break
    if count%100 ==0:
        print(count)
    count=count+1
    
cap.release()
out.release()
print("done")


connection_all size is not 19 : return
connection_all size is not 19 : return
0
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is 

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:103: RuntimeWarning: invalid value encountered in true_divide
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:123: RuntimeWarning: invalid value encountered in greater


connection_all size is not 19 : return
900
connection_all size is not 19 : return
connection_all size is not 19 : return
1000
1100
1200
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
1300
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
connection_all size is not 19 : return
c

In [85]:
fighters = [np.array([[ 2. ,  6.  ,  0.14019634,  1.   ,  2.        ],
       [ 3.        ,  7.        ,  0.10505158,  2.        ,  3.        ]]), np.array([[  2.        ,  10.        ,   0.15756657,   1.        ,   0.        ],
       [  3.        ,  11.        ,   0.13177204,   2.        ,   1.        ]])]
print(len(fighters))
print(fighters)

2
[array([[ 2.        ,  6.        ,  0.14019634,  1.        ,  2.        ],
       [ 3.        ,  7.        ,  0.10505158,  2.        ,  3.        ]]), array([[  2.        ,  10.        ,   0.15756657,   1.        ,   0.        ],
       [  3.        ,  11.        ,   0.13177204,   2.        ,   1.        ]])]
